In [23]:
import re
import pandas as pd
import requests as re
from bs4 import BeautifulSoup

In [13]:
headers = {"User-Agent": "mayankbohra.dev@gmail.com"}

In [14]:
def cik_matching_ticker(ticker, headers=headers):
    ticker = ticker.upper().replace(".", "-")
    try:
        ticker_json = re.get(
            "https://www.sec.gov/files/company_tickers.json", headers=headers
        ).json()
        
        for company in ticker_json.values():
            if company["ticker"] == ticker:
                cik = str(company["cik_str"]).zfill(10)
                return cik
        raise ValueError(f"Ticker {ticker} not found in SEC database")
    except Exception as e:
        print("An error occurred:", e)

In [15]:
cik_matching_ticker("aapl")

'0000320193'

In [16]:
def get_submission_data_for_ticker(ticker, only_filings_df=False):
    cik = cik_matching_ticker(ticker)
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    company_json = re.get(url, headers=headers).json()
    if only_filings_df:
        return pd.DataFrame(company_json['filings']['recent'])
    return company_json

In [17]:
def get_filtered_filings(ticker):
    company_filings_df = get_submission_data_for_ticker(ticker, only_filings_df=True)
    df = company_filings_df[company_filings_df['form'] == '10-K']
    
    df = df.set_index('reportDate')
    accession_df = df['accessionNumber']
    
    return accession_df

In [19]:
filings = get_filtered_filings("aapl")
filings

reportDate
2023-09-30    0000320193-23-000106
2022-09-24    0000320193-22-000108
2021-09-25    0000320193-21-000105
2020-09-26    0000320193-20-000096
2019-09-28    0000320193-19-000119
2018-09-29    0000320193-18-000145
2017-09-30    0000320193-17-000070
2016-09-24    0001628280-16-020309
2015-09-26    0001193125-15-356351
2014-09-27    0001193125-14-383437
Name: accessionNumber, dtype: object

In [22]:
acc = get_filtered_filings("aapl")
acc_num = acc.iloc[0].replace("-", "")
acc_num

'000032019323000106'